In [36]:
import json, os
import pandas as pd
from tqdm.notebook import tqdm
from neo4j import GraphDatabase
# from neo4j_genai.indexes import create_vector_index
import numpy as np
import re

In [37]:
with open(r"D:\BrainyBlaze\BB_Repos\access\neo4j.json", 'r') as f:
    neo4j_creds = json.load(f)

if 0:   # cloud base
    pass
    # URI = neo4j_creds['url']
    # AUTH = (neo4j_creds['user'], neo4j_creds['password'])
    # Database=neo4j_creds['database']
else:  # local
    URI = "bolt://localhost:7687"
    AUTH = ('neo4j', neo4j_creds['password'])
    Database='mm-lessons'  # manual ravgptv4 neo4j
    
VECTOR_INDEX_NAME = "lessons_text_embd"  # Part
# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH, max_connection_pool_size=25)
driver.verify_connectivity()
print("Connection established.")

Connection established.


In [3]:
folder = r"D:\Programming\My_reps\Similarity_search_audio\tmp_work_files\texts_from_audio"
embd_folder = r"D:\Programming\My_reps\Similarity_search_audio\tmp_work_files\embeddings_for_texts_openai"

In [4]:
with open(os.path.join(folder, 'uploaded_files.txt'), 'r') as f:
    uploaded_files = [p.replace('\n', '') for p in f.readlines()]
    
def update_processed(filename):
    global uploaded_files
    uploaded_files.append(os.path.basename(filename))
    with open(os.path.join(folder, 'uploaded_files.txt'), 'a') as f:
        f.writelines(uploaded_files[-1 ]+ '\n')

len(uploaded_files)

50

In [9]:
filename = "Рав Элиягу Бережинский. Рамбам , Пиркей Авот (8). #09-09-2024# [aN02ju-GWRk].csv"

In [10]:
re.search(r"(\[[^\]]+\]).csv$", filename).group(1)

'[aN02ju-GWRk]'

In [11]:
re.search(r" #([^#]+)# ", filename).group(1)

'09-09-2024'

In [5]:
# whole Lesson summarization
cur_file = tqdm([f for f in os.listdir(folder) if f.endswith('csv')])
for filename in cur_file:
    cur_file.set_description(filename[:40])
    youtube_link = re.search(r"(\[[^\]]+\]).csv$", filename).group(1)
    if youtube_link in uploaded_files:
        continue

    upload_date = re.search(r" #([^#]+)# ", filename).group(1)
    file_path = os.path.join(folder, filename)
    ds = pd.read_csv(file_path)
    text = ds.loc[:, 'text'].tolist()
    embeddings = np.load(os.path.join(embd_folder, os.path.splitext(filename)[0] + '.text.npy'))

    name = os.path.splitext(filename)[0].split(' #')[0]
    properties = {"name": name, "rav": None}

    lesson_node = driver.execute_query("CREATE (a:Lesson) SET a += $props RETURN a", props=properties, database_=Database)
    lesson_element_id = lesson_node[0][0]['a'].element_id

    for idx in range(len(text)):
        if len(text[idx].strip()) == 0:
            continue
        properties = {"lesson_name": name, 'part': idx + 1, "embeddings": embeddings[idx], "text": text[idx], 
                      'time_start': ds.loc[idx, 'start'], "time_end": ds.loc[idx, 'end'], "youtube_id": youtube_link[1:-1],
                     "upload_date": upload_date, "_date_sort": int(''.join(upload_date.split('-')[::-1]))}
        
        summary = driver.execute_query("""CREATE (txt:Part)
                SET txt += $props
                    WITH txt
                    MATCH (les)
                    WHERE elementId(les) = $les_element_id
                    WITH les, txt
                    CREATE (les)-[:Text_part]->(txt)
                    """, props=properties, les_element_id=lesson_element_id, 
                                       database_=Database)
    
    update_processed(youtube_link)

  0%|          | 0/847 [00:00<?, ?it/s]

In [30]:
ds

,start,end,text,summarization
0,0.00,177.00,мы с вами открываем 14 псалом и давайте скажем...,В ходе урока рассматривается 14 псалом Давида....
1,177.00,411.40,"Очень интересно. Я, к сожалению, в музыке не о...","В тексте обсуждается поведение человека, котор..."
2,411.40,537.78,"И если вы скажете, что не может быть такого, ч...",В тексте обсуждается тема праведников и их рол...
3,537.78,713.68,"Если человек бесчестный, отвратительный, говор...",В тексте обсуждается важность интеллекта и пон...
4,713.68,876.96,"она дошла до того, я не знаю, как тут сказать,...",В тексте обсуждается история борьбы с определё...
5,876.96,1068.10,земли так всегда происходит уравнение до уравн...,В тексте обсуждается опасность нивелирования г...
6,1068.10,1258.64,которые равны то есть естественно что этот спо...,"В тексте обсуждается опасность прогрессивизма,..."
7,1258.64,1440.96,"который призывал Всевышнего, искал Всевышнего,...",В тексте обсуждается проблема понимания собств...
8,1440.96,1615.82,"Ну и, может быть, и правильно делаем, иногда в...",В тексте обсуждается сложность выбора между те...
9,1615.82,1798.12,"То, что в Шаваке называется «Малката райот». К...",В тексте обсуждается тема самообвинения под пы...


---

# Create vector index

# CLEAR DATABASE

In [38]:
db = 'manual'

In [46]:
driver.execute_query("match (a) -[r] -> () DELETE a, r", database_=db)
driver.execute_query("match (a) delete a", database_=db)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000014AC58816F0>, keys=[])